In [1]:
from tqdm import tqdm
import argparse
import random
from test_func import *
from client import *
from utils import *

import json

from dotenv import load_dotenv
import os

# Load the .env file
load_dotenv()

# -*- coding: utf-8 -*-

# parser = argparse.ArgumentParser()
# parser.add_argument("--dataset", type=str,
#                     default="webqsp", help="choose the dataset.")
# parser.add_argument("--max_length", type=int,
#                     default=256, help="the max length of LLMs output.")
# parser.add_argument("--temperature_exploration", type=float,
#                     default=0.4, help="the temperature in exploration stage.")
# parser.add_argument("--temperature_reasoning", type=float,
#                     default=0, help="the temperature in reasoning stage.")
# parser.add_argument("--width", type=int,
#                     default=3, help="choose the search width of ToG.")
# parser.add_argument("--depth", type=int,
#                     default=3, help="choose the search depth of ToG.")
# parser.add_argument("--remove_unnecessary_rel", type=bool,
#                     default=True, help="whether removing unnecessary relations.")
# parser.add_argument("--LLM_type", type=str,
#                     default="gpt-3.5-turbo", help="base LLM model.")
# parser.add_argument("--opeani_api_keys", type=str,
#                     default="", help="if the LLM_type is gpt-3.5-turbo or gpt-4, you need add your own openai api keys.")
# parser.add_argument("--num_retain_entity", type=int,
#                     default=5, help="Number of entities retained during entities search.")
# parser.add_argument("--prune_tools", type=str,
#                     default="llm", help="prune tools for ToG, can be llm (same as LLM_type), bm25 or sentencebert.")
# parser.add_argument("--addr_list", type=str,
#                     default="server_urls.txt", help="The address of the Wikidata service.")
# args = parser.parse_args()

args_dict = {
    "dataset": "myself",
    "max_length": 256,
    "temperature_exploration": 0.4,
    "temperature_reasoning": 0,
    "width": 3,
    "depth": 3,
    "remove_unnecessary_rel": True,
    "LLM_type": "gpt-4o-mini",
    "opeani_api_keys": os.getenv("OPENAI_API_KEY"),
    "num_retain_entity": 5,
    "prune_tools": "llm",
    "addr_list": "F:\\CodingEnvironment\\tog\\ToG\\server_urls.txt"
}

args = argparse.Namespace(**args_dict)

    
datas, question_string = prepare_dataset(args.dataset)
print("Start Running ToG on %s dataset." % args.dataset)
for data in tqdm(datas):
    question = data[question_string]
    topic_entity = data['qid_topic_entity']
    cluster_chain_of_entities = []
    if len(topic_entity) == 0:
        results = generate_without_explored_paths(question, args)
        save_2_jsonl(question, results, [], file_name=args.dataset)
        continue
    pre_relations = []
    pre_heads= [-1] * len(topic_entity)
    flag_printed = False

    # with open(args.addr_list, "r") as f:
    #     server_addrs = f.readlines()
    #     server_addrs = [addr.strip() for addr in server_addrs]
    # print(f"Server addresses: {server_addrs}")
    # wiki_client = MultiServerWikidataQueryClient(server_addrs)

    import json

    file_path = r'F:\CodingEnvironment\tog\fci_graph.json'
    with open(file_path, 'r', encoding='utf-8', errors='ignore') as file:
        graph = json.load(file)

    for depth in range(1, args.depth+1):
        current_entity_relations_list = []
        i=0
        for entity in topic_entity:
            if entity!="[FINISH_ID]":
                retrieve_relations_with_scores = relation_search_prune(entity, pre_relations, pre_heads[i], question, args, graph)  # best entity triplet, entitiy_id
                current_entity_relations_list.extend(retrieve_relations_with_scores)
            i+=1
        total_candidates = []
        total_scores = []
        total_relations = []
        total_entities_id = []
        total_topic_entities = []
        total_head = []

        print(current_entity_relations_list)

        raise Exception("stop")

Start Running ToG on myself dataset.


  0%|          | 0/1 [00:00<?, ?it/s]

Please retrieve 3 relations (separated by semicolon) that contribute to the question and rate their contribution on a scale from 0 to 1 (the sum of the scores of 3 relations is 1).
Q: Mesih Pasha's uncle became emperor in what year?
Topic Entity: Mesih Pasha
Relations:
1. wiki.relation.child
2. wiki.relation.country_of_citizenship
3. wiki.relation.date_of_birth
4. wiki.relation.family
5. wiki.relation.father
6. wiki.relation.languages_spoken, written_or_signed
7. wiki.relation.military_rank
8. wiki.relation.occupation
9. wiki.relation.place_of_death
10. wiki.relation.position_held
11. wiki.relation.religion_or_worldview
12. wiki.relation.sex_or_gender
13. wiki.relation.sibling
14. wiki.relation.significant_event
A: 1. {wiki.relation.family (Score: 0.5)}: This relation is highly relevant as it can provide information about the family background of Mesih Pasha, including his uncle who became emperor.
2. {wiki.relation.father (Score: 0.4)}: Uncle is father's brother, so father might provi

  0%|          | 0/1 [00:02<?, ?it/s]

1. {wiki.relation.affiliation (Score: 0.5)}: This relation is highly relevant as it can provide information about the organization or department where Vũ Hải Yến works, which may help identify the deputy director of her division.
2. {wiki.relation.position_held (Score: 0.4)}: This relation is relevant because it can provide insights into the specific roles and titles held by Vũ Hải Yến, potentially leading to information about her deputy director.
3. {wiki.relation.employer (Score: 0.1)}: This relation is moderately relevant as it can give context about the organization Vũ Hải Yến is associated with, but may not directly lead to identifying the deputy director.
[{'entity': 'Vũ Hải Yến', 'relation': 'affiliation', 'score': 0.5, 'head': False}, {'entity': 'Vũ Hải Yến', 'relation': 'position held', 'score': 0.4, 'head': False}, {'entity': 'Vũ Hải Yến', 'relation': 'employer', 'score': 0.1, 'head': False}]


Exception: stop

In [2]:
from tqdm import tqdm
import argparse
import random
from test_func import *
from client import *
from utils import *

#duyet lai ten entity
for entity in current_entity_relations_list:
    value_flag=False
    
    entity_candidates = entity_search(entity['entity'], entity['relation'], graph, entity['head'])

    scores, entity_candidates = entity_score(question, entity_candidates, entity['score'], entity['relation'], args)
    
    total_candidates, total_scores, total_relations, total_topic_entities, total_head = update_history(entity_candidates, entity, scores, total_candidates, total_scores, total_relations, total_topic_entities, total_head, value_flag)

# raise Exception("stop")

if len(total_candidates) ==0:
    half_stop(question, cluster_chain_of_entities, depth, args)
    flag_printed = True
    # break #@
    raise Exception("stop")
    
flag, chain_of_entities, pre_relations, pre_heads = entity_prune( total_relations, total_candidates, total_topic_entities, total_head, total_scores, args, graph)
cluster_chain_of_entities.append(chain_of_entities)
if flag:
    stop, results = reasoning(question, cluster_chain_of_entities, args)
    if stop:
        print("ToG stoped at depth %d." % depth)
        save_2_jsonl(question, results, cluster_chain_of_entities, file_name=args.dataset)
        flag_printed = True
        # break #@
        raise Exception("stop")
    else:
        print("depth %d still not find the answer." % depth)
        # flag_finish, entities_id = if_finish_list(entities_id)
        # if flag_finish:
        #     half_stop(question, cluster_chain_of_entities, depth, args)
        #     flag_printed = True
        # else:

        # topic_entity = {qid: topic for qid, topic in zip(entities_id, [wiki_client.query_all("qid2label", entity).pop() for entity in entities_id])}
        # continue #@
        raise Exception("stop")
else:
    half_stop(question, cluster_chain_of_entities, depth, args)
    flag_printed = True

raise Exception("finish")

depth 1 still not find the answer.


Exception: stop

In [3]:
print(cluster_chain_of_entities)

[[[('affiliation', 'Trung tâm sản phẩm Conversation'), ('position held', 'Trung tâm sản phẩm Conversation'), ('employer', 'Trung tâm sản phẩm Conversation')]]]


In [4]:
print(entity_candidates)

['Trung tâm sản phẩm Conversation']


In [5]:
total_topic_entities

['Vũ Hải Yến', 'Vũ Hải Yến', 'Vũ Hải Yến']